In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import urllib.request
import numpy as np
import re


### **함수 preprocess_text** <br>
입력 데이터 전처리 작업:
한글, 숫자, 영어를 제외한 글자는 모두 '_'로 대치

In [3]:
def preprocess_text(text):
   
    # input  text ='이 SF영화 재미^^;; 없는데.....왜 7점이지???'
    # output text ='$이_SF영화_재미_없는데_왜_7점이지$'
    
    non_alpha_numeric_hangul = re.compile('[^0-9a-zA-Z\u3131-\u3163\uac00-\ud7a3]')
    SS = '$'  # Sentence Separator
    
    t = non_alpha_numeric_hangul.sub('_', text)
    text = re.sub(r'_+', '_', t)
    return SS+text+SS

### **함수 prepare_data_file** <br>

입력 파일 형식에 맞추어 data와 target 값 읽어오기

---

**id	document	label**<br>
6270596	굳 ㅋ	1 <br>
9274899	GDNTOPCLASSINTHECLUB	0 <br>
8544678	뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아	0<br>
6825595	지루하지는 않은데 완전 막장임... 돈주고 보기에는....	0

In [4]:
def prepare_data_file(FILE_PATH):
    
    # file load from URL
    with urllib.request.urlopen(FILE_PATH) as f:
        lines = f.read().decode('utf-8').split('\n')

    data, target = [], []
    for l in lines[1:]:   # 첫번째 줄은 skip
        try:
            _, text, label = l.strip().split('\t')   # ID\treview\tlabel 
        except ValueError:
            pass
        text = text.strip()
        if text == '': continue
        data.append(preprocess_text(text))
        target.append(int(label))
            
    return data, target

### **함수 extract_features** <br>
Bigram feature 추출하여 feature_dict로 만들어 return <br>
MAX_FEATURES: 사용할 feature 갯수
```
$이_SF영화_재미_없는데_왜_7점이지$ 
($이) (이_) (_S) (SF) (F영) (영화) (화_) (_재) (재미) .. (이지) (지$)
```


In [5]:
def extract_features(data, MAX_FEATURES):
    
    FEATURES = dict()
      
    for line in data:
        uni_list = list(line)
        bi_list = [''.join(uni_list[z:z+2]) for z in range(0, len(uni_list)-1)]
        for bigram in bi_list:
            if bigram in FEATURES:
                FEATURES[bigram] += 1
            else:
                FEATURES[bigram] = 1
    
    features_list = [(x, f) for (x, f) in FEATURES.items()]
    features_list.sort(reverse=True, key=lambda z:z[1])
    features_dict = dict()
    for (idx, (x, y)) in enumerate(features_list[:MAX_FEATURES]):
        features_dict[x] = idx

    return features_dict

### **함수 make_feature_vevtor** <br>
입력 문장을 고정된 크기의 Feature Vector로 변환<br>

```
$이_SF영화_재미_없는데_왜_7점이지$ 
($이) (이_) (_S) (SF) (F영) (영화) (화_) (_재) (재미) .. (이지) (지$)
[1, 0, 0, 1, 0, 0, ...., 1, 0, 0]
```


In [6]:
def make_feature_vector(feature_set, data, target):

    fv_base = [0 for _ in range(0, len(feature_set))]
    feature_list = []
    for (x, label) in zip(data, target):
        uni_list = list(x)
        fv = fv_base[:]
        bi_list = [''.join(uni_list[z:z+2]) for z in range(0, len(uni_list)-1)]
        for bigram in bi_list: 
            if bigram in feature_set:
                fv[feature_set[bigram]] = 1
        feature_list.append(fv + [label])
    feature_list = np.array(feature_list)
    np.random.shuffle(feature_list)
    
    return feature_list[:, :-1], feature_list[:, -1]

### **학습데이터, 평가데이터 읽어오기**

In [7]:
# NAVER MOVIE REVIEW corpus로부터 데이터 loading
TRAIN_FILE = 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt'
TEST_FILE  = 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt'

print('prepare_data_file START...')
train_data, train_target = prepare_data_file(TRAIN_FILE)
test_data, test_target = prepare_data_file(TEST_FILE)
print('prepare_data_file END...')


prepare_data_file START...
prepare_data_file END...


### **학습데이터로부터 MAX_FEATURES 개의 bigram feature 추출**

In [8]:
# Bigram Feature 추출
print('extract_features START...')
MAX_FEATURES = 1000  # 사용할 feature 개수
feature_set = extract_features(train_data, MAX_FEATURES)
with open('/content/drive/MyDrive/IntroAI/features.out', 'w', encoding='utf8') as fo:
    fo.write('\n'.join([x+'\t'+str(idx) for x, idx in feature_set.items()]))
print('extract_features END...')

extract_features START...
extract_features END...


### **입력 파일을 고정된 크기의 feature vector로 변화**

In [9]:
# 입력 파일을 feature vector로 변환
print('make_feature_vector START...')
x_train, y_train = make_feature_vector(feature_set, train_data, train_target)
x_test,  y_test  = make_feature_vector(feature_set, test_data, test_target)
print('make_feature_vector END...')

make_feature_vector START...
make_feature_vector END...


### **기계학습 모델 생성 및 학습**

In [15]:
# Machine Learning Tool을 사용하여 학습

from sklearn.naive_bayes import MultinomialNB

print('train START...')
model = MultinomialNB()    # Naive Baysian
model.fit(x_train, y_train)
print('train END...')


train START...
train END...


### **정확도 평가**

In [16]:
# Model의 정확도 평가
print('eval START...')
print("훈련 세트 정확도: {:.3f}".format(model.score(x_train, y_train)))  # Eval
print("테스트 세트 정확도: {:.3f}".format(model.score(x_test, y_test)))  # Eval
print('eval END...')

eval START...
훈련 세트 정확도: 0.781
테스트 세트 정확도: 0.778
eval END...


In [10]:
############# (4) Logistic Regression 

from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(max_iter=500)
logreg.fit(x_train, y_train)

LogisticRegression(max_iter=500)

In [11]:
# Model의 정확도 평가
print('eval START...')
print("훈련 세트 정확도: {:.3f}".format(logreg.score(x_train, y_train)))  # Eval
print("테스트 세트 정확도: {:.3f}".format(logreg.score(x_test, y_test)))  # Eval
print('eval END...')

eval START...
훈련 세트 정확도: 0.804
테스트 세트 정확도: 0.799
eval END...


### **수행해 보기**

In [12]:
# 수행해 보기
text = '최악의 영화?'   
text = preprocess_text(text)
x_test, _ = make_feature_vector(feature_set, [text], [None])
result = logreg.predict(x_test)
print(text, '==> ', ['Negative', 'Positive'][result[0]])

$최악의_영화_$ ==>  Negative
